In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
%matplotlib inline

# Using Divvy data

## Load and concat the Divvy datasets

In [ ]:
# divvy1 = pd.read_csv('data/Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q1.csv', parse_dates=['start_time', 'end_time'])
# divvy2 = pd.read_csv('data/Divvy_Trips_2017_Q1Q2/Divvy_Trips_2017_Q2.csv', parse_dates=['start_time', 'end_time'])
# divvy3 = pd.read_csv('data/Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q3.csv', parse_dates=['start_time', 'end_time'])
# divvy4 = pd.read_csv('data/Divvy_Trips_2017_Q3Q4/Divvy_Trips_2017_Q4.csv', parse_dates=['start_time', 'end_time'])

# divvy = pd.concat([divvy1, divvy2, divvy3, divvy4], ignore_index=True)

# divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

In [ ]:
divvy

In [ ]:
station_list = set(list(divvy['from_station_name'].unique()) + list(divvy['to_station_name'].unique()))
station_list

In [ ]:
len(station_list)

## Load Divvy station info for GPS coordinates

In [ ]:
with open('data/stations.json') as json_data:
    station_data = json.load(json_data)

In [ ]:
station_data['stationBeanList']

In [ ]:
stations = [station['stationName'] for station in station_data['stationBeanList']]
latitude = [station['latitude'] for station in station_data['stationBeanList']]
longitude = [station['longitude'] for station in station_data['stationBeanList']]

In [ ]:
len(stations)

In [ ]:
unknown = []
for station in station_list:
    if station not in stations:
        unknown.append(station)
unknown

In [ ]:
station_gps = pd.DataFrame({'station_name': stations, 'latitude': latitude, 'longitude': longitude})

In [ ]:
station_gps.head()

In [ ]:
def gps_lookup(location):
    match = (station_gps['station_name'] == location)
    coord = station_gps['latitude'][match]
    if len(coord) > 0:
        return pd.Series([coord.values[0], station_gps['longitude'][match].values[0]])
    else:
        return pd.Series([np.nan, np.nan])

In [ ]:
divvy[['from_station_latitude', 'from_station_longitude']] = divvy['from_station_name'].apply(gps_lookup)
divvy[['to_station_latitude', 'to_station_longitude']] = divvy['to_station_name'].apply(gps_lookup)

In [ ]:
divvy

In [ ]:
divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

### 1) Top 5 stations with the most starts (showing # of starts)

In [ ]:
station_starts = divvy.groupby(['from_station_name'])['from_station_name'].count().sort_values(ascending=False)
station_starts.head()

In [ ]:
ax = station_starts.head(5).plot(kind='bar', figsize=(15, 10), title='Top 5 Stations with Most Starts')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .15, p.get_height() * 1.005))

### 2) Trip duration by user type

In [ ]:
trip_duration = divvy.groupby(['usertype'])['tripduration'].mean().sort_values(ascending=False)
trip_duration

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10))

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10), showfliers=False)

### 3) Most popular trips based on start station and stop station

In [ ]:
divvy['trip_stations'] = divvy['from_station_name'] + ' TO ' + divvy['to_station_name']

In [ ]:
trip_stations = divvy.groupby(['trip_stations'])['trip_stations'].count().sort_values(ascending=False)
trip_stations.head()

In [ ]:
ax = trip_stations.head(10).plot(kind='bar', figsize=(15, 10), title='Top 10 Most Popular Trips')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .05, p.get_height() * 1.005))

### Create dataframe of paths for Tableau chart

In [ ]:
trip_dict = {'path': list(trip_stations.index), 'frequency': list(trip_stations.values)}
trip_dict['origin'] = [x.split(' TO ')[0] for x in trip_dict['path']]
trip_dict['destination'] = [x.split(' TO ')[1] for x in trip_dict['path']]

In [ ]:
from collections import defaultdict

trips = defaultdict(list)

for idx in range(len(trip_dict['path'])):
    trips['path'].append(trip_dict['path'][idx])
    trips['frequency'].append(trip_dict['frequency'][idx])
    trips['origin-destination'].append('origin')
    trips['station'].append(trip_dict['origin'][idx])
    
    trips['path'].append(trip_dict['path'][idx])
    trips['frequency'].append(trip_dict['frequency'][idx])
    trips['origin-destination'].append('destination')
    trips['station'].append(trip_dict['destination'][idx])

In [ ]:
stations_gps = (divvy[divvy['from_station_name'].duplicated()]
                [['from_station_name', 'from_station_latitude', 'from_station_longitude']].reset_index(drop=True))

In [ ]:
trips_df = pd.DataFrame(trips).merge(stations_gps.drop_duplicates(subset=['from_station_name']),
                                     how='left',
                                     left_on='station',
                                     right_on='from_station_name')

In [ ]:
trips_df.drop(['from_station_name'], axis=1, inplace=True)

In [ ]:
trips_df.to_csv('data/trips.csv')

### 4) Rider performance by Gender and Age based on avg trip distance (station to station), median speed (distance traveled / trip duration)

Multiply geodesic distance by 1.25. Routes follow roads but the calculated route is direct (geodesic). A route straight down a road would be the same as the direct route; a route diagnoal to roads would be multiplied by 1.414 (thanks, Pythagoras!); assuming routes are evenly split between diagonal and direct, with some wiggle room, I'm splitting the difference at 1.25.

I looked at using the Google Maps api to calculate the actual, along-the-road distance, but they've removed the free api key option. I also looked at Bing Maps, but it's rate limited and I have more than 98,000 routes in this dataset (and once I saw how big that number was, I realized that using api calls would take more than a few days!). So I opted for this *x1.25* method which is less accurate but far quicker and cheaper.

In [ ]:
from math import isnan

def find_distance(row):
    if (not isnan(row['from_station_latitude']) and
        not isnan(row['from_station_longitude']) and
        not isnan(row['to_station_latitude']) and
        not isnan(row['to_station_longitude'])):
        return (1.25 * (geopy.distance.distance((row['from_station_latitude'], row['from_station_longitude']),
                                                (row['to_station_latitude'], row['to_station_longitude'])).m))
    return np.nan

In [ ]:
from math import isnan

def find_distance(row):
    if (not isnan(row['from_station_latitude']) and
        not isnan(row['from_station_longitude']) and
        not isnan(row['to_station_latitude']) and
        not isnan(row['to_station_longitude'])):
        distance = (1.25 * (geopy.distance.distance((row['from_station_latitude'], row['from_station_longitude']),
                                                (row['to_station_latitude'], row['to_station_longitude'])).m))
        print(distance)
        return distance
    else:
        print(np.nan)
    return np.nan

In [ ]:
divvy['trip_distance'] = divvy.apply(find_distance, axis=1)
divvy['speed'] = divvy['trip_distance'] / divvy['tripduration']

In [ ]:
divvy.to_csv('data/divvy_2017.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017.csv')

# Yelp data

In [ ]:
business = pd.read_json('data/yelp_dataset/yelp_academic_dataset_business.json', lines=True)

checkin = pd.read_json('data/yelp_dataset/yelp_academic_dataset_checkin.json', lines=True)

max_records = 1e5
df = pd.read_json('data/yelp_dataset/yelp_academic_dataset_review.json', lines=True, chunksize=max_records)
review = pd.DataFrame() # Initialize the dataframe
try:
    for df_chunk in df:
        review = pd.concat([review, df_chunk])
except ValueError:
    print ('\nSome messages in the file cannot be parsed')

In [ ]:
business

In [ ]:
checkin

In [ ]:
review

In [ ]:
checkin['checkins'] = checkin['time'].apply(lambda x : sum(x.values()))
yelp = business.merge(checkin, on='business_id', how='outer')

In [ ]:
business.to_csv('data/yelp_business.csv', index=False)
checkin.to_csv('data/yelp_checkin.csv', index=False)
review.to_csv('data/yelp_review.csv', index=False)
yelp.to_csv('data/yelp.csv', index=False)

In [ ]:
review['date'] = pd.to_datetime(review['date'])
review[review['date'].dt.year == 2017]

### Find the top 10 and bottom 10 restaurants in Illinois having most and least checkins respectively.

In [ ]:
yelp['checkins'] = yelp['time'].apply(lambda x : sum(x.values()))

In [ ]:
checkin['checkins'] = checkin['time'].apply(lambda x : sum(x.values()))

In [ ]:
yelp.head()

In [ ]:
len(yelp[(yelp['state'] == 'IL') &
         (yelp['categories'].str.contains('Restaurants')) &
         (yelp['checkins'].isnull()) &
         (yelp['is_open'] == 1)])

In [ ]:
len(yelp[(yelp['state'] == 'IL') &
         (yelp['categories'].str.contains('Restaurants')) &
         (yelp['checkins'] == 1) &
         (yelp['is_open'] == 1)])

In [ ]:
len(yelp[(yelp['state'] == 'IL') &
         (yelp['categories'].str.contains('Restaurants')) &
         (yelp['checkins'] == 2) &
         (yelp['is_open'] == 1)])

In [ ]:
len(yelp[(yelp['state'] == 'IL') &
         (yelp['categories'].str.contains('Restaurants')) &
         (yelp['checkins'] == 3) &
         (yelp['is_open'] == 1)])

In [ ]:
top10 = yelp[(yelp['state'] == 'IL') &
             (yelp['categories'].str.contains('Restaurants') &
              (yelp['is_open'] == 1))
            ].sort_values('checkins', ascending=False).head(10)

bottom10 = yelp[(yelp['state'] == 'IL') &
                (yelp['categories'].str.contains('Restaurants') &
                (yelp['is_open'] == 1))
               ].sort_values('checkins', ascending=False).tail(10)

In [ ]:
top10

In [ ]:
bottom10

In [ ]:
top10.to_csv('data/yelp_top10_checkins.csv', index=False)
bottom10.to_csv('data/yelp_bottom10_checkins.csv', index=False)

In [ ]:
top10_names = top10['name'].tolist()
top10_names

In [ ]:
bottom10_names = bottom10['name'].tolist()
bottom10_names

### For the top 10 and bottom 10 restaurants calculated in step 6, calculate the average star rating and average sentiment score of the reviews

In [ ]:
top_bottom = top10['business_id'].tolist() + bottom10['business_id'].tolist()
top_bottom

In [ ]:
sentiment = review[review['business_id'].isin(top_bottom)]

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def get_vader_scores(text):
    '''
    Takes a string of text and outputs four values for Vader's negative,
    neutral, positive, and compound (normalized) sentiment scores
    INPUT: a string
    OUTPUT: a dictionary of four sentiment scores
    '''

    analyser = SentimentIntensityAnalyzer()
    return analyser.polarity_scores(text)


def apply_vader(df, column):
    '''
    Takes a DataFrame with a specified column of text and adds four new columns
    to the DataFrame, corresponding to the Vader sentiment scores
    INPUT: DataFrame, string
    OUTPUT: the original DataFrame with four additional columns
    '''

    sentiment = pd.DataFrame(df[column].apply(get_vader_scores))
    unpacked = pd.DataFrame([d for idx, d in sentiment['text'].iteritems()],
                            index=sentiment.index)
    unpacked['compound'] += 1
    columns = {'neu': 'v_neutral', 'pos': 'v_positive', 'neg': 'v_negative'}
    unpacked.rename(columns=columns, inplace=True)
    return pd.concat([df, unpacked], axis=1)

In [ ]:
sentiment = apply_vader(sentiment, 'text')

In [ ]:
sentiment

In [ ]:
top10_sentiment = {}
for biz_id in top10['business_id'].tolist():
    top10_sentiment[yelp.loc[yelp['business_id'] == biz_id, 'name'].iloc[0]] = sentiment[sentiment['business_id'] == biz_id].groupby(['business_id']).mean()

In [ ]:
top10_scores = pd.DataFrame()
for restaurant in top10_sentiment:
    top10_scores = top10_scores.append(pd.DataFrame(top10_sentiment[restaurant]))
top10_scores.reset_index(level=0, inplace=True)

In [ ]:
top10_scores['name'] = top10_scores['business_id'].apply(lambda business_id: yelp['name']
                                                         [(yelp['business_id'] == business_id)].values[0])

In [ ]:
top10_scores

In [ ]:
bottom10_sentiment = {}
for biz_id in bottom10['business_id'].tolist():
    bottom10_sentiment[yelp.loc[yelp['business_id'] == biz_id, 'name'].iloc[0]] = sentiment[sentiment['business_id'] == biz_id].groupby(['business_id']).mean()

In [ ]:
bottom10_scores = pd.DataFrame()
for restaurant in bottom10_sentiment:
    bottom10_scores = bottom10_scores.append(pd.DataFrame(bottom10_sentiment[restaurant]))
bottom10_scores.reset_index(level=0, inplace=True)

In [ ]:
bottom10_scores['name'] = bottom10_scores['business_id'].apply(lambda business_id: yelp['name']
                                                               [(yelp['business_id'] == business_id)].values[0])

In [ ]:
bottom10_scores

In [ ]:
top10_scores.to_csv('data/yelp_top_scores.csv', index=False)
bottom10_scores.to_csv('data/yelp_bottom_scores.csv', index=False)

### What are the top 10 Cuisine types (Mexican, American, Thai, etc) based on the number of restaurants and number of check ins

In [ ]:
restaurants = yelp[yelp['categories'].str.contains('Restaurants', na=False)]

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'
restaurants['categories'] = restaurants['categories'].apply(lambda x: x.split(', '))

In [ ]:
restaurants = restaurants.join(pd.get_dummies(pd.DataFrame(restaurants['categories'].tolist()).stack()).astype(int).sum(level=0))

In [ ]:
columns = [item for item in restaurants.columns.tolist() if item not in yelp.columns.tolist()]
columns.remove('Restaurants')

In [ ]:
num_restaurants = restaurants[columns].sum(numeric_only=True).sort_values(ascending=False)
num_restaurants

In [ ]:
checkin_dict ={}
for item in columns:
    cnt = restaurants[restaurants[item] == 1].groupby([item])['checkins'].sum()
    if cnt.empty:
        checkin_dict[item] = 0
    else:
        checkin_dict[item] = restaurants[restaurants[item] == 1].groupby([item])['checkins'].sum().values[0]

In [ ]:
checkin_dict = pd.Series(checkin_dict).sort_values(ascending=False)

In [ ]:
checkin_dict

In [ ]:
for item in checkin_dict.keys():
    print(item)

In [ ]:
pd.DataFrame(checkin_dict).to_csv('data/yelp_cuisine_checkin.csv')
pd.DataFrame(num_restaurants).to_csv('data/yelp_cuisine_restaurants.csv',)

### What are the most popular keywords or adjectives that reviewers use for the above list of cuisines (calculated in step 8)

In [ ]:
cuisine_list = ['American (Traditional)', 'Mexican', 'Chinese', 'Italian', 'American (New)',
               'Japanese', 'Thai', 'Mediterranean', 'Asian Fusion', 'Indian', 'Korean',
                'Greek', 'Mddle Eastern', 'Vietnamese', 'French', 'Tex-Mex', 'Caribbean']
cuisines = pd.DataFrame()
for cuisine in cuisine_list:
    businesses = yelp[yelp['categories'].str.contains(cuisine, na=False)]['business_id'].tolist()
    temp = review[review['business_id'].isin(businesses)]
    temp['cuisine'] = cuisine
    cuisines = pd.concat([cuisines, temp])

In [ ]:
cuisines['date'] = pd.to_datetime(cuisines['date'])

In [ ]:
cuisines_2017 = cuisines[cuisines['date'].dt.year == 2017]

In [ ]:
cuisines = cuisines_2017.sample(frac=0.05)

In [ ]:
cuisine_docs = {}
for cuisine in cuisine_list:
    cuisine_docs[cuisine] = cuisines[cuisines['cuisine'] == cuisine]['text'].str.cat(sep=' ')

In [ ]:
import spacy
from collections import Counter

nlp = spacy.load('en')

for cuisine in cuisine_list:
    doc = nlp(cuisine_docs[cuisine])
    # all tokens that arent stop words or punctuations
    words = [token.text for token in self.doc if token.is_stop != True and token.is_punct != True]

    # noun tokens that arent stop words or punctuations
    nouns = [token.text for token in self.doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"]

    # five most common tokens
    word_freq = Counter(words)
    common_words = word_freq.most_common(5)

    # five most common noun tokens
    noun_freq = Counter(nouns)
    common_nouns = noun_freq.most_common(5)

In [ ]:
import spacy
from collections import Counter

nlp = spacy.load('en')
nlp.max_length=3000000
doc = nlp(cuisine_docs['Mexican'])
# all tokens that arent stop words or punctuations
words = [token.text for token in self.doc if token.is_stop != True and token.is_punct != True]

# noun tokens that arent stop words or punctuations
nouns = [token.text for token in self.doc if token.is_stop != True and token.is_punct != True and token.pos_ == "NOUN"]

# five most common tokens
word_freq = Counter(words)
common_words = word_freq.most_common(5)

# five most common noun tokens
noun_freq = Counter(nouns)
common_nouns = noun_freq.most_common(5)

In [ ]:
for cuisine in cuisine_list:
    print(cuisine, len(cuisine_docs[cuisine]) / 1000000)

# Using Kaggle data

In [ ]:
# divvy = pd.read_csv('data/chicago-divvy-bicycle-sharing-data/data_raw.csv', parse_dates=['starttime', 'stoptime'])

In [ ]:
# divvy = divvy[divvy['starttime'].dt.year == 2017].reset_index(drop=True)

In [ ]:
# divvy.to_csv('data/divvy_2017_kaggle.csv')

In [ ]:
divvy = pd.read_csv('data/divvy_2017_kaggle.csv')

In [ ]:
divvy

In [ ]:
divvy.head()

In [ ]:
divvy.columns

### 1) Top 5 stations with the most starts (showing # of starts)

In [ ]:
station_starts = divvy.groupby(['from_station_name'])['from_station_name'].count().sort_values(ascending=False)
station_starts.head()

In [ ]:
ax = station_starts.head(5).plot(kind='bar', figsize=(15, 10), title='Top 5 Stations with Most Starts')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .15, p.get_height() * 1.005))

In [ ]:
station_coord = 

In [ ]:
map = Basemap(width=10000000,height=6000000,projection='lcc',
            resolution=None,lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
plt.figure(figsize=(19,20))
map.bluemarble()

In [ ]:
for station in station_starts[5]:
        loc = geolocator.geocode(city)
        if not loc:
            print("Could not locate {}".format(city))
            continue
        x, y = map(loc.longitude, loc.latitude)
        map.plot(x,y,marker='o',color='Red',markersize=int(math.sqrt(count))*scale)
        plt.annotate(city, xy = (x,y), xytext=(-20,20)) 

### 2) Trip duration by user type

In [ ]:
trip_duration = divvy.groupby(['usertype'])['tripduration'].mean().sort_values(ascending=False)
trip_duration

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10))

In [ ]:
ax = divvy.boxplot(column='tripduration', by='usertype', figsize=(15,10), showfliers=False)

### 3) Most popular trips based on start station and stop station

In [ ]:
divvy['trip_stations'] = divvy['from_station_name'] + ' TO ' + divvy['to_station_name']

In [ ]:
trip_stations = divvy.groupby(['trip_stations'])['trip_stations'].count().sort_values(ascending=False)
trip_stations.head(10)

In [ ]:
ax = trip_stations.head(10).plot(kind='bar', figsize=(15, 10), title='Top 10 Most Popular Trips')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005 + .05, p.get_height() * 1.005))

In [ ]:
trip_dict = {'path': list(trip_stations.index), 'frequency': list(trip_stations.values)}
trip_dict['origin'] = [x.split(' TO ')[0] for x in trip_dict['path']]
trip_dict['destination'] = [x.split(' TO ')[1] for x in trip_dict['path']]

In [ ]:
from collections import defaultdict

trips = defaultdict(list)

for idx in range(len(trip_dict['path'])):
    trips['path'].append(trip_dict['path'][idx])
    trips['origin-destination'].append('origin')
    trips['station'].append(trip_dict['origin'][idx])
    
    trips['path'].append(trip_dict['path'][idx])
    trips['origin-destination'].append('destination')
    trips['station'].append(trip_dict['destination'][idx])

In [ ]:
stations_gps = (divvy[divvy['from_station_name'].duplicated()]
                [['from_station_name', 'latitude_start', 'longitude_start']].reset_index(drop=True))

In [ ]:
trips_df = pd.DataFrame(trips).merge(stations_gps, how='left', left_on='station', right_on='from_station_name')

In [ ]:
trips_df.to_csv('data/trips.csv')

### 4) Rider performance by Gender and Age based on avg trip distance (station to station), median speed (distance traveled / trip duration)

Multiply geodesic distance by 1.25. Routes follow roads but the calculated route is direct (geodesic). A route straight down a road would be the same as the direct route; a route diagnoal to roads would be multiplied by 1.414 (thanks, Pythagoras!); assuming routes are evenly split between diagonal and direct, with some wiggle room, I'm splitting the difference at 1.25.

I looked at using the Google Maps api to calculate the actual, along-the-road distance, but they've removed the free api key option. I also looked at Bing Maps, but it's rate limited and I have more than 98,000 routes in this dataset (and once I saw how big that number was, I realized that using api calls would take more than a few days!). So I opted for this *x1.25* method which is less accurate but far quicker and cheaper.

In [ ]:
def find_distance(row):
    return (1.25 * (geopy.distance.distance((row['latitude_start'], row['longitude_start']),
                                            (row['latitude_end'], row['longitude_end'])).m))

In [ ]:
divvy['trip_distance'] = divvy.apply(find_distance, axis=1)

In [ ]:
divvy['speed'] = divvy['trip_distance'] / divvy['tripduration']

In [ ]:
divvy[['starttime', 'stoptime', 'tripduration', 'latitude_start', 'longitude_start', 'latitude_end', 'longitude_end', 'trip_distance', 'speed']]